In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0

In [0]:
import os.path
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelBinarizer


root_dir_train = '/content/drive/My Drive/DataSet Desafio BB/data/train'
root_dir_test = '/content/drive/My Drive/DataSet Desafio BB/data/test'


def getimagedataandlabels(root_dir, image_w, image_h):

    X_data=[]
    Y_data=[]
    classes_from_directories = []  # to determine the classes from the root folder structure automatically

    for directory, subdirectories, files in os.walk(root_dir):
        for subdirectory in subdirectories:
            classes_from_directories.append(subdirectory)
        for file in files:
            if file != '.DS_Store':  # fix for MAC...
                imagepath = os.path.join(directory, file)
                current_image_class_splitt = imagepath.split('/')
                current_image_class = current_image_class_splitt[len(current_image_class_splitt) - 2]
                img = cv2.imread (imagepath)
                X_data.append(np.asarray(img, dtype="int32"))
                Y_data.append(current_image_class)
               

    return np.array(X_data), np.array(Y_data)

#Loading the training images
x_train, y_train = getimagedataandlabels(root_dir_train, 256, 256)
print("Training images and labels loaded")
print(x_train.shape)
print(y_train.shape)

#Loading the te
x_test, y_test = getimagedataandlabels(root_dir_test, 256, 256)
print("Test images and labels loaded")
print(x_test.shape)
print(y_test.shape)


#Brining the labels into the correct format for categorical crossentropy
encoder = LabelBinarizer()
Y_train = encoder.fit_transform(y_train)
Y_test = encoder.fit_transform(y_test)

In [0]:
import tensorflow as tf
import keras
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', input_shape=(256, 256, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
model.add(tf.keras.layers.Dropout(rate=0.05))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
model.add(tf.keras.layers.Dropout(rate=0.05))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=64, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(rate=0.10))
model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(rate=0.10))
model.add(tf.keras.layers.Dense(units=5, activation='softmax'))
model.summary()


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

list_callbacks = [
    ModelCheckpoint(filepath='best_model.hdf5', monitor='val_accuracy', verbose=1, mode='max', save_best_only=True),
    EarlyStopping(monitor='val_accuracy', patience=1000, verbose=1, mode='max', restore_best_weights=True)
]

model.fit(x_train / 255, Y_train, 
          batch_size=8, 
          epochs=1000, 
          validation_data=(x_test / 255,Y_test),
          callbacks=list_callbacks)